In [ ]:
from sklearn.cluster import AffinityPropagation
from sklearn import metrics
from sklearn.datasets.samples_generator import make_blobs

In [ ]:
import os

In [ ]:
directory = "C:\\Users\\44070779\\Desktop\\docu"

In [ ]:
f_names = []
filenames = os.listdir(directory)
for file in filenames:
    f_name = os.path.join(directory, file)
    f_names.append(f_name)

In [ ]:
data_corpus = {}

for f in filenames:
    f_name = os.path.join(directory, f)
    fi = open(f_name)
    data = fi.read()
    data_corpus[f] = data

In [ ]:
def strip(raw_text):
    letters_only = re.sub('[\s+]', " ", raw_text)
    return letters_only

In [ ]:
import pandas as pd
import re

In [ ]:
data = pd.DataFrame.from_dict(data_corpus,orient='index')

In [ ]:
data = data.reset_index()
data.columns = ['file_name','text']

In [ ]:
data['strip']= data['text'].apply(strip)

In [ ]:
import nltk
import string

In [ ]:
def remove_punctuation(s):
    s = ''.join([i for i in s if i not in frozenset(string.punctuation)])
    return s

data['text_punc'] = data['strip'].apply(remove_punctuation)

In [ ]:
def numbers(raw_text):
    letters_only = re.sub(r'\d+', '', raw_text)
    return letters_only

In [ ]:
data['num'] = data['text_punc'].apply(numbers)

In [ ]:
from nltk import pos_tag

In [ ]:
words = set(nltk.corpus.words.words())
def non_eng_text(raw_text):
    wor = raw_text.split()
    meaningful_words = [w for w in wor \
         if w.lower() in words or not w.isalpha()]
    return( " ".join( meaningful_words ))

In [ ]:
data['non_eng']= data['num'].apply(non_eng_text)

In [ ]:
data['lower'] = data['non_eng'].str.lower()

In [ ]:
def clean_text(raw_text):
    letters_only = re.sub("[^a-z]", " ", raw_text)
    words = letters_only.split()
    corp = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours',
'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers',
'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves',
'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are',
'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does',
'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until',
'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into',
'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down',
'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here',
'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',
'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so',
'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', 'should', 'now']
    stops = set(corp)
    meaningful_words = [w for w in words if not w in stops and len(w)>1]
    return( " ".join( meaningful_words ))

In [ ]:
data['stop']= data['lower'].apply(clean_text)

In [ ]:
from nltk import pos_tag

In [ ]:
from nltk import word_tokenize, pos_tag
from nltk.stem import WordNetLemmatizer
def lemma(raw_text):
    wor = raw_text.split()
    wnl = WordNetLemmatizer()
    meaningful = [wnl.lemmatize(i,j[0].lower()) if j[0].lower() in ['a','n','v'] else wnl.lemmatize(i) for i,j in pos_tag(wor)]
    return ( " ".join( meaningful))

In [ ]:
data['text_lemma']= data['stop'].apply(lemma)

In [ ]:
corp = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours',
'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers',
'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves',
'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are',
'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does',
'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until',
'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into',
'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down',
'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here',
'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',
'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so',
'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', 'should', 'now']
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=500000, stop_words = corp, use_idf= True, ngram_range=(1,3))
%time train_data_feature=tfidf_vectorizer.fit_transform(data['text_lemma'])
print(train_data_feature.shape)

In [ ]:
tfidf_feature_names = tfidf_vectorizer.get_feature_names()

In [ ]:
tf_vectorizer = CountVectorizer( max_features=500000, stop_words = corp)
tf = tf_vectorizer.fit_transform(data['text_lemma'])
tf_feature_names = tf_vectorizer.get_feature_names()

In [ ]:
import sklearn

In [ ]:
train_data_norm = sklearn.preprocessing.normalize(train_data_feature)
tf_norm = sklearn.preprocessing.normalize(tf)

In [ ]:
train_data_np = train_data_norm.toarray()
tf_np = tf_norm.toarray()

In [ ]:
sum(train_data_np[1])

In [ ]:
X, labels_true = make_blobs(cluster_std=0.5,
                            random_state=0)


In [ ]:
af = AffinityPropagation(preference=-50).fit(train_data_feature)
cluster_centers_indices = af.cluster_centers_indices_
labels = af.labels_

n_clusters_ = len(cluster_centers_indices)

print('Estimated number of clusters: %d' % n_clusters_)
print("Homogeneity: %0.3f" % metrics.homogeneity_score(labels_true, labels))
print("Completeness: %0.3f" % metrics.completeness_score(labels_true, labels))
print("V-measure: %0.3f" % metrics.v_measure_score(labels_true, labels))
print("Adjusted Rand Index: %0.3f"
      % metrics.adjusted_rand_score(labels_true, labels))
print("Adjusted Mutual Information: %0.3f"
      % metrics.adjusted_mutual_info_score(labels_true, labels))
print("Silhouette Coefficient: %0.3f"
      % metrics.silhouette_score(X, labels, metric='sqeuclidean'))

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
dist = 1 - cosine_similarity(train_data_feature)


In [ ]:
import matplotlib.pyplot as plt


In [ ]:
titles = open('NAME.txt').read().split('\n')

In [ ]:
from scipy.cluster import hierarchy

In [ ]:
from scipy.cluster.hierarchy import ward, dendrogram

linkage_matrix = ward(dist) #define the linkage_matrix using ward clustering pre-computed distances

fig, ax = plt.subplots(figsize=(15, 20)) # set size
ax = dendrogram(linkage_matrix, orientation="right", labels=titles);

plt.tick_params(\
    axis= 'x',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom='off',      # ticks along the bottom edge are off
    top='off',         # ticks along the top edge are off
    labelbottom='off')

plt.tight_layout() #show plot with tight layout

#uncomment below to save figure
plt.savefig('ward_clusters.png', dpi=200) #save figure as ward_clusters


In [ ]:
plt.close()

In [ ]:
import markup

In [ ]:
import tempfile